In [1]:
import pandas as pd

test = pd.read_parquet('data/test.parquet')

In [2]:
import numpy as np

# Функция для преобразования массива значений в список чисел (если это необходимо)
def convert_to_float_list(values):
    if isinstance(values, str):  # Если данные представлены как строка
        return list(map(float, values.strip('[]').split(',')))
    elif isinstance(values, np.ndarray):  # Если данные уже массив
        return values.tolist()
    else:
        return values  # Оставляем как есть, если это уже список

# Применяем преобразование к 'values'
test['values'] = test['values'].apply(convert_to_float_list)

In [7]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis

# Функция для генерации признаков на основе временного ряда
def generate_features(data):
    features = pd.DataFrame()
    features['id'] = data['id']

    # Рассчитываем статистические признаки
    features['mean'] = data['values'].apply(np.mean)
    features['std'] = data['values'].apply(np.std)  # Оставляем std вместо variance
    features['min'] = data['values'].apply(np.min)
    features['max'] = data['values'].apply(np.max)
    features['median'] = data['values'].apply(np.median)

    # Дополнительные статистические признаки
    features['kurtosis'] = data['values'].apply(lambda x: kurtosis(x) if len(x) > 1 else 0)  # Куртозис

    # Автокорреляция с лагом 1
    features['autocorr'] = data['values'].apply(lambda x: pd.Series(x).autocorr(lag=1))

    # Размах (Range)
    features['range'] = features['max'] - features['min']


    # Количество пиков (локальные максимумы)
    features['peaks'] = data['values'].apply(lambda x: len(pd.Series(x).diff().fillna(0).ge(0).astype(int).diff().fillna(0).lt(0)))
    # Лаговые признаки
    for lag in range(1, 4):
        features[f'lag_{lag}'] = data['values'].apply(lambda x: x[-lag] if len(x) > lag else np.nan)

    # Скользящие статистики
    features['rolling_mean'] = data['values'].apply(lambda x: pd.Series(x).rolling(window=3).mean().iloc[-1])
    features['rolling_std'] = data['values'].apply(lambda x: pd.Series(x).rolling(window=3).std().iloc[-1])

    return features

# Генерация признаков для тестовой выборки

test_features = generate_features(test)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [4]:
!pip install catboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00


In [5]:
from catboost import CatBoostClassifier

# Создание экземпляра модели
loaded_model = CatBoostClassifier()

# Загрузка модели
loaded_model.load_model('model/catboost_model.cbm')

In [8]:
# Предсказание на тестовых данных
test_preds = loaded_model.predict_proba(test_features.drop(columns=['id']))[:, 1]

# Создаем DataFrame для submission
submission = pd.DataFrame({
    'id': test_features['id'],
    'score': test_preds
})

# Сохраняем в submission.csv
submission.to_csv('submission.csv', index=False)
print('Файл submission.csv успешно сохранен!')

Файл submission.csv успешно сохранен!
